In [2]:
pip install pandas requests beautifulsoup4 tqdm


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime
import time
import random

# ===== 1. File đầu vào =====
input_file = "dantri_new_unique_2025-10-12_00-50-08.csv"
df = pd.read_csv(input_file)

# Kiểm tra cột link
if "link" not in df.columns:
    raise ValueError(" File CSV không có cột 'link', không thể cào được!")

links = df["link"].dropna().unique().tolist()
print(f" Tổng số link cần cào: {len(links)}")

# ===== 2. Hàm cào từng bài báo =====
def crawl_article(url):
    try:
        headers = {"User-Agent": "Mozilla/5.0"}
        res = requests.get(url, headers=headers, timeout=15)
        if res.status_code != 200:
            return None
        
        soup = BeautifulSoup(res.text, "html.parser")

        # Tiêu đề (nếu muốn ghi đè)
        title_tag = soup.find("h1")
        title = title_tag.get_text(strip=True) if title_tag else ""

        # Ngày đăng
        time_tag = soup.find("span", class_="author-time")
        if not time_tag:
            time_tag = soup.find("time")
        pub_date = time_tag.get_text(strip=True) if time_tag else ""

        # Tác giả
        author = ""
        author_tag = soup.find("div", class_="author-name")
        if author_tag:
            author = author_tag.get_text(strip=True)
        else:
            strong = soup.find("p", string=lambda s: s and "Tác giả" in s)
            author = strong.get_text(strip=True) if strong else ""

        # Nội dung
        content_div = soup.find("div", class_="singular-content")
        if not content_div:
            content_div = soup.find("article")
        paragraphs = content_div.find_all("p") if content_div else []
        content = "\n".join(p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True))

        return {
            "link": url,
            "title": title,
            "pub_date": pub_date,
            "author": author,
            "content": content,
            "crawl_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }

    except Exception:
        return None

# ===== 3. Cào dữ liệu từng link =====
results = []
for url in tqdm(links, desc=" Đang cào nội dung bài báo"):
    data = crawl_article(url)
    if data:
        results.append(data)
    time.sleep(random.uniform(0.5, 1.2))  # nghỉ nhẹ tránh bị chặn

# ===== 4. Gộp dữ liệu =====
if not results:
    print("\n Không có dữ liệu được cào.")
else:
    df_crawl = pd.DataFrame(results)

    # Gộp theo 'link' để nối nội dung với dữ liệu cũ
    df_final = pd.merge(df, df_crawl, on="link", how="left")

    # Lưu file mới
    output_file = f"dantri_crawl_full_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.csv"
    df_final.to_csv(output_file, index=False, encoding="utf-8-sig")

    print(f"\n Hoàn tất! Đã cào {len(df_crawl)} bài.")
    print(f" Dữ liệu được lưu trong file: {output_file}")


 Tổng số link cần cào: 7580


 Đang cào nội dung bài báo: 100%|██████████| 7580/7580 [4:06:37<00:00,  1.95s/it]  



 Hoàn tất! Đã cào 7580 bài.
 Dữ liệu được lưu trong file: dantri_crawl_full_2025-10-13_17-27-40.csv
